<a href="https://colab.research.google.com/github/sunshineluyao/ESG_TwoSources/blob/main/Sorting/Abnormal_Return_MSCI_2020_1_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Creat the Leader, Average, and Laggard Group

In [17]:
import pandas as pd
import numpy as np

In [18]:
df =pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Sorting/Benchmark.csv',index_col='Unnamed: 0')
df.head()

,Symbol,Month,Size,Mom,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Monthly_Return,Portfoio_Return
0,A,1,26461.7,11.6,-0.128462,5,3.0,2.0,5.32,-3.2,-0.791667
1,AVB,1,29287.1,12.5,-0.210955,5,3.0,2.0,5.32,3.3,-0.791667
2,ECL,1,55628.3,16.3,-0.170082,5,3.0,2.0,5.32,1.6,-0.791667
3,ICE,1,51536.5,15.2,-0.171353,5,3.0,2.0,5.32,7.8,-0.791667
4,IQV,1,29980.8,16.7,-0.124864,5,3.0,2.0,5.32,0.5,-0.791667


In [19]:
df['Abnormal_Return']=df['Monthly_Return']-df['Portfoio_Return']
df.head()

,Symbol,Month,Size,Mom,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Monthly_Return,Portfoio_Return,Abnormal_Return
0,A,1,26461.7,11.6,-0.128462,5,3.0,2.0,5.32,-3.2,-0.791667,-2.408333
1,AVB,1,29287.1,12.5,-0.210955,5,3.0,2.0,5.32,3.3,-0.791667,4.091667
2,ECL,1,55628.3,16.3,-0.170082,5,3.0,2.0,5.32,1.6,-0.791667,2.391667
3,ICE,1,51536.5,15.2,-0.171353,5,3.0,2.0,5.32,7.8,-0.791667,8.591667
4,IQV,1,29980.8,16.7,-0.124864,5,3.0,2.0,5.32,0.5,-0.791667,1.291667


In [20]:
ESG = pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Sorting/Separate_ESG_Score.csv')
ESG.head()

,Symbol,Name,Stock Exchange,RBICS Economy,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance
0,A,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9
1,AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,B,2.3,3.4,3.1,3.7
2,AAN,Aaron's Company Inc,NYSE,Consumer Cyclicals,@NA,NaN,NaN,NaN,NaN
3,AAOI,"Applied Optoelectronics, Inc.",NASDAQ,Technology,@NA,NaN,NaN,NaN,NaN
4,AAON,"AAON, Inc.",NASDAQ,Industrials,A,7.1,5.5,6.5,5.5


In [21]:
ESG.isna().sum()

Symbol                                                   0
Name                                                     0
Stock Exchange                                           0
RBICS Economy                                            0
MSCI ESG Rtg Overall Company Rating                      0
MSCI ESG Rtg Overall Industry-Adjusted Company Score    65
MSCI ESG Rtg Score Environmental                        65
MSCI ESG Rtg Score Social                               65
MSCI ESG Rtg Score Governance                           65
dtype: int64

In [22]:
ESG=ESG.dropna()

In [23]:
ESG['ESG_G']=pd.qcut(ESG['MSCI ESG Rtg Overall Industry-Adjusted Company Score'],5,labels=np.arange(1,6,1))
ESG['E_G']=pd.qcut(ESG['MSCI ESG Rtg Score Environmental'],5,labels=np.arange(1,6,1))
ESG['S_G']=pd.qcut(ESG['MSCI ESG Rtg Score Social'],5,labels=np.arange(1,6,1))
ESG['G_G']=pd.qcut(ESG['MSCI ESG Rtg Score Governance'],5,labels=np.arange(1,6,1))

In [24]:
ESG['ESG_Group']=ESG['ESG_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['E_Group']=ESG['E_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['S_Group']=ESG['S_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['G_Group']=ESG['G_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')

In [25]:
df=df.merge(ESG, on=['Symbol'])
df.head()

,Symbol,Month,Size,Mom,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Monthly_Return,Portfoio_Return,Abnormal_Return,Name,Stock Exchange,RBICS Economy,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance,ESG_G,E_G,S_G,G_G,ESG_Group,E_Group,S_Group,G_Group
0,A,1,26461.7,11.6,-0.128462,5,3.0,2.0,5.32,-3.2,-0.791667,-2.408333,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average
1,A,2,25608.7,26.5,-0.128462,5,3.0,2.0,5.32,-6.6,-8.516667,1.916667,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average
2,A,3,23950.9,8.6,-0.128462,5,3.0,2.0,5.32,-7.1,-15.541667,8.441667,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average
3,A,4,22502.3,-5.1,-0.096675,5,3.0,1.0,5.31,11.2,11.600000,-0.400000,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average
4,A,5,23737.9,-12.2,-0.096675,5,3.0,2.0,5.32,18.8,8.675000,10.125000,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average


# ESG: Visualizing Abnormal Return Monthly

In [26]:
import plotly.express as px
fig = px.box(df, x="ESG_Group", y="Abnormal_Return",color='ESG_Group',animation_frame='Month',animation_group='Symbol')
fig.show()

In [27]:
ESG_summary = df.groupby(by=['Month','ESG_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
ESG_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
ESG_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
ESG_summary.head()

,Month,ESG_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.110339,8.635907,847,29.103264,0.296733
1,1,Laggard,0.838670,8.648286,287,16.941074,0.510492
2,1,Leader,0.117235,7.417502,277,16.643317,0.445674
3,2,Average,-0.075168,8.824186,847,29.103264,0.303203
4,2,Laggard,0.376053,9.231752,287,16.941074,0.544933


In [28]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

In [29]:
ESG_summary = ESG_summary[ESG_summary['ESG_Group'].isin(['Leader', 'Laggard'])]

In [30]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

### E result

In [31]:
E_summary = df.groupby(by=['Month','E_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
E_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
E_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
E_summary.head()

,Month,E_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.751761,8.683248,857,NaN,NaN
1,1,Laggard,-2.012759,8.415027,284,16.941074,0.510492
2,1,Leader,1.088866,7.044592,270,16.643317,0.445674
3,2,Average,0.124371,9.049209,857,NaN,NaN
4,2,Laggard,-1.060744,7.780479,284,16.941074,0.544933


In [32]:
E_summary = E_summary[E_summary['E_Group'].isin(['Leader', 'Laggard'])]

In [33]:
import plotly.express as px
fig = px.line(E_summary, x="Month", y="mean", color="E_Group", error_y="mean_std")
fig.show()

### S result

In [34]:
S_summary = df.groupby(by=['Month','S_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
S_summary['sqrt_count']=S_summary['count'].apply(lambda x: np.sqrt(x))
S_summary['mean_std']=S_summary['std']/S_summary['sqrt_count']
S_summary.head()

,Month,S_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.467788,8.547254,854,29.223278,0.292481
1,1,Laggard,-0.033759,8.058334,307,17.521415,0.459913
2,1,Leader,-0.089989,8.385666,250,15.811388,0.530356
3,2,Average,-0.021274,8.570508,854,29.223278,0.293277
4,2,Laggard,0.030820,8.864521,307,17.521415,0.505925


In [36]:
S_summary = S_summary[S_summary['S_Group'].isin(['Leader', 'Laggard'])]

In [37]:
import plotly.express as px
fig = px.line(S_summary, x="Month", y="mean", color="S_Group", error_y="mean_std")
fig.show()

### G result

In [38]:
G_summary = df.groupby(by=['Month','G_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
G_summary['sqrt_count']=G_summary['count'].apply(lambda x: np.sqrt(x))
G_summary['mean_std']=G_summary['std']/G_summary['sqrt_count']
G_summary.head()

,Month,G_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.502416,7.992707,834,28.879058,0.276765
1,1,Laggard,0.490785,8.887571,308,17.549929,0.506416
2,1,Leader,-0.756681,9.056756,269,16.401219,0.552200
3,2,Average,0.023469,8.489377,834,28.879058,0.293963
4,2,Laggard,0.344484,10.219875,308,17.549929,0.582331


In [40]:
G_summary = G_summary[G_summary['G_Group'].isin(['Leader', 'Laggard'])]

In [41]:
import plotly.express as px
fig = px.line(G_summary, x="Month", y="mean", color="G_Group", error_y="mean_std")
fig.show()

# Step 3: Conduct Two-group t-test for each month

In [42]:
pip install pingouin

     |████████████████████████████████| 225kB 5.2MB/s 
  Created wheel for pingouin: filename=pingouin-0.3.9-cp36-none-any.whl size=223375 sha256=d31f997acdb7762e0c26121783f56608aa3af5fdb4b17738ee78b3b7b5ae7d40
  Stored in directory: /root/.cache/pip/wheels/6c/d7/26/a23b9ed74061adc5cf8f5e016bbe2902a8ebf63b6471047dbc
  Created wheel for outdated: filename=outdated-0.2.0-cp36-none-any.whl size=4961 sha256=483a830a84dad46757cccd88efe8a9a4eddfe4846e884fed2b1d0f4808fade83
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp36-none-any.whl size=7051 sha256=0c91c70c3dc9045bc5e04cfff7a79dbd2d93a84c20c3a67ef08857c2cf5bafa8
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built pingouin outdated littleutils


In [43]:
import pingouin as pg


Two-Sample T-Test

https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

### Test Monthly the Leaders' Abornormal Return >0?

In [ ]:
Leader = df[(df['Month']==1) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.509725,262,two-sided,0.012686,"[0.24, 2.0]",0.154756,1.501,0.705596


In [ ]:
df_ = pd.DataFrame()
Month_List =df.Month.unique().tolist()
for i in Month_List:
  Leader =  df[(df['Month']==i) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
  t_test = pg.ttest(Leader, 0)
  t_test['Month']=i
  df_=df_.append(t_test)

In [ ]:
df_

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,2.509725,262,two-sided,0.012686,"[0.24, 2.0]",0.154756,1.501,0.705596,1
T-test,-0.258230,262,two-sided,0.796432,"[-1.0, 0.77]",0.015923,0.071,0.057617,2
T-test,1.508329,262,two-sided,0.132675,"[-0.47, 3.56]",0.093008,0.212,0.324044,3
T-test,-1.269640,262,two-sided,0.205339,"[-3.12, 0.67]",0.078289,0.153,0.244165,4
T-test,0.139721,262,two-sided,0.888988,"[-1.21, 1.39]",0.008616,0.07,0.052223,5
T-test,1.007838,262,two-sided,0.314462,"[-0.47, 1.47]",0.062146,0.114,0.171100,6
T-test,1.897563,262,two-sided,0.058853,"[-0.05, 2.74]",0.117009,0.405,0.472411,7
T-test,1.210042,262,two-sided,0.227353,"[-0.45, 1.9]",0.074614,0.142,0.226092,8
T-test,0.649735,262,two-sided,0.516433,"[-0.67, 1.34]",0.040064,0.085,0.099220,9
T-test,1.329981,262,two-sided,0.184681,"[-0.36, 1.84]",0.082010,0.165,0.263271,10


In [ ]:
df_2 = pd.DataFrame()
Month_List =df.Month.unique().tolist()
for i in Month_List:
  Leader = df[(df['Month']==i) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
  Laggard = df[(df['Month']==i) & (df['ESG_Group']=='Laggard')]['Abnormal_Return']
  t_test = pg.ttest(Leader, Laggard,paired=False)
  t_test['Month']=i
  df_2=df_2.append(t_test)

In [ ]:
df_2

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,0.950945,534.684344,two-sided,0.342062,"[-0.72, 2.08]",0.080049,0.147,0.155202,1
T-test,-1.927577,483.624501,two-sided,0.054493,"[-3.25, 0.03]",0.160775,0.577,0.469503,2
T-test,0.924248,548.731295,two-sided,0.355764,"[-1.5, 4.18]",0.078593,0.144,0.151307,3
T-test,-0.625961,539.395695,two-sided,0.531606,"[-3.92, 2.02]",0.052763,0.115,0.094781,4
T-test,0.314840,542.642122,two-sided,0.753004,"[-1.49, 2.05]",0.026859,0.1,0.061415,5
T-test,1.298513,549.055022,two-sided,0.194656,"[-0.49, 2.39]",0.109956,0.215,0.251384,6
T-test,2.688456,528.308735,two-sided,0.007405,"[0.68, 4.34]",0.230290,3.149,0.769626,7
T-test,2.513743,539.500461,two-sided,0.012236,"[0.44, 3.62]",0.214679,2.032,0.710468,8
T-test,1.430712,529.843145,two-sided,0.153102,"[-0.36, 2.29]",0.122509,0.257,0.300164,9
T-test,2.340982,546.027499,two-sided,0.019592,"[0.32, 3.65]",0.197826,1.355,0.639588,10


# Overall test

In [44]:
Leader =df[df['ESG_Group']=='Leader']['Abnormal_Return']
Laggard=df[df['ESG_Group']=='Laggard']['Abnormal_Return']

In [ ]:
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.243726,2892,two-sided,0.024925,"[0.06, 0.86]",0.041715,0.259,0.611431


In [45]:
t_test = pg.ttest(Leader, Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,3.455697,6074.979077,two-sided,0.000553,"[0.43, 1.56]",0.087502,10.998,0.931079


In [46]:
E_Leader =df[df['E_Group']=='Leader']['Abnormal_Return']
E_Laggard=df[df['E_Group']=='Laggard']['Abnormal_Return']

In [49]:
t_test = pg.ttest(E_Leader, E_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.561041,5959.737732,two-sided,0.01046,"[0.18, 1.33]",0.065331,0.76,0.721732


In [47]:
S_Leader =df[df['S_Group']=='Leader']['Abnormal_Return']
S_Laggard=df[df['S_Group']=='Laggard']['Abnormal_Return']

In [51]:
t_test = pg.ttest(S_Leader, S_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.653207,5779.16632,two-sided,0.007995,"[0.21, 1.38]",0.068428,0.968,0.759015


In [48]:
G_Leader =df[df['G_Group']=='Leader']['Abnormal_Return']
G_Laggard=df[df['G_Group']=='Laggard']['Abnormal_Return']

In [52]:
t_test = pg.ttest(G_Leader, G_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.196714,6265.667896,two-sided,0.028077,"[0.07, 1.26]",0.055187,0.315,0.592137


In [ ]:
df.to_csv("ESG_MSCI_FINAL.csv")